In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xarray as xr
import pingouin as pg
import os
from tqdm import tqdm
from os.path import join as pjoin
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import pearsonr, spearmanr, zscore, wilcoxon
import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:
sys.path.append('..')
import circletrack_neural as ctn
import circletrack_behavior as ctb
import pca_ica as ica
import plotting_functions as pf

In [24]:
## Create a list of sessions for easy plotting later and for changing column order
session_list = ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1']
## Set path variables
dpath = '/media/caishuman/csstorage/phild/git/MazeProjects/output/assemblies'
behav_path = '/media/caishuman/csstorage/phild/git/MazeProjects/output/behav'
## Create young and old mouse list
young_mice = ['Fornax', 'Janus', 'Lyra', 'Miranda', 'Naiad', 'Sao', 'Titania']
old_mice = ['Gemini', 'Oberon', 'Puck', 'Rhea', 'Umbriel', 'Virgo', 'Ymir', 'Atlas']
## Create list of female or male young mice
young_female_mice = ['Fornax', 'Janus', 'Lyra', 'Sao', 'Titania']
male_young_mice = ['Miranda', 'Naiad']
## Set x_bin_size for binning by time (in seconds)
x_bin_size = 5

The code block below bins the data by trials and determines ensemble trends.

In [20]:
## Set mouse list, path, and z_thresh for binarization
across_time = False
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
mouse_trial_times = {}
## Loop through each mouse
for mouse in tqdm(young_mice):
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    trial_times = {}
    ## Load assemblies
    for session in session_list:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, dpath, session)
        ## Set activation values as act
        act = assemblies.activations.values
        ## Load a specific session's behavior data
        if not across_time:
            behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
            aligned_behavior = pd.read_feather(behav_file)
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = aligned_behavior.trials
            ## Get length of time for each trial
            time_diff = []
            for trial in np.unique(trials):
                ## Subset aligned_behavior by a given trial
                behavior = aligned_behavior.loc[trials == trial]
                ## Get the first and last timestamp to determine the window
                first_timestamp, last_timestamp = behavior.t.to_numpy()[0], behavior.t.to_numpy()[-1]
                ## Convert from ms to s
                first_timestamp = first_timestamp / 1000
                last_timestamp = last_timestamp / 1000
                ## Append to time_diff list
                time_diff.append(last_timestamp - first_timestamp)
            trial_times[session] = time_diff
        ## This is where the data gets binned either by even time intervals or by trials
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold = None, x_bin_size = x_bin_size, analysis_type = 'max', zscored = True, alpha = 'sidak')  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'all', z_threshold = None)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends
    mouse_trial_times[mouse] = trial_times

 14%|█▍        | 1/7 [00:03<00:19,  3.24s/it]/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning:

divide by zero encountered in log1p

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning:

divide by zero encountered in log1p

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning:

divide by zero encountered in log1p

 29%|██▊       | 2/7 [00:06<00:15,  3.12s/it]/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning:

divide by zero encountered in log1p

 43%|████▎     | 3/7 [00:09<00:11,  2.96s/it]/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning:

divide by zero encountered in log1p

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-pack

In [ ]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
mouse_name = []
for key in mouse_trends:
    prop_decreasing_T4.append(mouse_trends[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_trends[key]['Reversal1']['prop_decreasing'])
    mouse_name.append(key)
prop_df = pd.DataFrame({'mouse': mouse_name, 'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
# prop_df = prop_df.drop(labels = 6, axis = 0)
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df.Training4.mean(), prop_df.Reversal1.mean()]
sem = [prop_df.Training4.sem(), prop_df.Reversal1.sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data.Training4, data.Reversal1], mode = 'lines', line_color = 'lightgrey', showlegend = False,
                             line = dict(width = 1), name = prop_df.loc[row, 'mouse']))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by Forward Trials',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
# fig.write_image('progress_figures/cohort0/by_trials.png')

In [ ]:
wilcoxon_result = wilcoxon(x = prop_df.Training4, y = prop_df.Reversal1, method = 'approx')
wilcoxon_result

In [21]:
## Plot fading ensembles across all sessions
## Create empty dictionary
trends_dict = {}
## Loop through mouse_trends and extract the proportion of fading ensembles for each session (Training1, Training2, etc)
for key in mouse_trends:
    fading_ensembles = []
    for session in mouse_trends[key]:
        fading_ensembles.append(mouse_trends[key][session]['prop_decreasing'])
    trends_dict[key] = fading_ensembles
## Plot values for each mouse
fig = go.Figure()
x_axis = session_list
for key in trends_dict:
    fig.add_trace(go.Scatter(x = x_axis, y = trends_dict[key], name = key))
fig.update_layout(template = 'simple_white', height = 500, width = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensemble Proportion Across Sessions',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.show()
# fig.write_image('progress_figures/cohort0/by_trials_all.png')

The code below determines ensemble trends by binning by even time intervals.

In [25]:
## Set mouse list, path, and z_thresh for binarization
across_time = True
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
## Loop through each mouse
for mouse in tqdm(young_mice):
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    ## Load assemblies
    for session in session_list:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, dpath, session)
        ## Set activation values as act
        act = assemblies.activations.values
        ## Load a specific session's behavior data
        if not across_time:
            behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
            aligned_behavior = pd.read_feather(behav_file)
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = aligned_behavior.trials
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold = None, x_bin_size = x_bin_size, analysis_type = 'max', zscored = True, alpha = 'sidak')  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'all', z_threshold = None)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends

100%|██████████| 7/7 [00:34<00:00,  4.94s/it]


In [26]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
mouse_name = []
for key in mouse_trends:
    prop_decreasing_T4.append(mouse_trends[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_trends[key]['Reversal1']['prop_decreasing'])
    mouse_name.append(key)
prop_df = pd.DataFrame({'mouse': mouse_name, 'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df.Training4.mean(), prop_df.Reversal1.mean()]
sem = [prop_df.Training4.sem(), prop_df.Reversal1.sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data.Training4, data.Reversal1], mode = 'lines', line_color = 'lightgrey', showlegend = False,
                             line = dict(width = 1), name = prop_df.loc[row, 'mouse']))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by {}s Bins'.format(x_bin_size),
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
# fig.write_image('progress_figures/cohort0/by_{}s_bins.png'.format(x_bin_size))

In [ ]:
wilcoxon_result = wilcoxon(x = prop_df.Training4, y = prop_df.Reversal1, method = 'approx')
wilcoxon_result

In [ ]:
## Plot fading ensembles across all sessions
## Create empty dictionary
trends_dict = {}
## Loop through mouse_trends and extract the proportion of fading ensembles for each session (Training1, Training2, etc)
for key in mouse_trends:
    fading_ensembles = []
    for session in mouse_trends[key]:
        fading_ensembles.append(mouse_trends[key][session]['prop_decreasing'])
    trends_dict[key] = fading_ensembles
## Plot values for each mouse
fig = go.Figure()
x_axis = session_list
for key in trends_dict:
    fig.add_trace(go.Scatter(x = x_axis, y = trends_dict[key], name = key))
fig.update_layout(template = 'simple_white', height = 500, width = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensemble Proportion Across Sessions',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.show()
fig.write_image('progress_figures/cohort0/by_{}s_bins_all.png'.format(x_bin_size))

Analyze average trial times.

In [ ]:
## Create a plot of average trial time across each session for all mice
df = pd.DataFrame(mouse_trial_times).T
mouse_avg_trial_times = {}
for mouse in df.index:
    session_dict = {}
    for session in df.columns:
        trial_times = df.loc[mouse, session]
        ## Bin trial time to see if trials are longer earlier in the session
        bin_size = 2
        if type(trial_times) is not list:
            pass
        else:
            ## Create bins
            bins = np.arange(1, len(trial_times), bin_size)
            ## Bin data
            binned = np.split(trial_times, bins)
            ## Take the average of each bin
            avg = [np.mean(bin) for bin in binned]
            session_dict[session] = avg
    mouse_avg_trial_times[mouse] = session_dict
## Avg time across each bin
session_averages = ctb.trial_averages(mouse_avg_trial_times, session_list)
avg_times = pd.DataFrame(mouse_avg_trial_times).T
avg_times = avg_times[session_list]
## Make figure
fig = make_subplots(rows = 2, cols = 3, subplot_titles = session_list, shared_yaxes = 'all')
for mouse in avg_times.index:
    for i, session in enumerate(avg_times.columns):
        data = avg_times.loc[mouse, session]
        x_axis = np.arange(0, len(data))
        if i <= 2:
            fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.6, showlegend = False), row = 1, col = i+1)
        elif (i > 2) & (i <= 5):
            fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.6, showlegend = False), row = 2, col = i-2)
for i, session in enumerate(session_list):
    data = session_averages[session]
    x_axis = np.arange(0, len(data))
    if i <= 2:
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', opacity = 0.7, showlegend = False), row = 1, col = i+1)
    else:
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', opacity = 0.7, showlegend = False), row = 2, col = i-2)
fig.update_layout(template = 'simple_white', width = 600, height = 600)
for col in np.arange(1, 4):
    fig.update_xaxes(title = 'Trial', row = 3, col = col)
    fig.update_yaxes(title = 'Trial Time (s)', row = col, col = 1)
fig.update_layout(
    title={
        'text': 'Binned Trial Time Across Session',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
for col in np.arange(1, 4):
    fig.update_xaxes(title = 'Trial', row = 3, col = col)
    fig.update_yaxes(title = 'Trial Time (s)', row = col, col = 1)
fig.show()
fig.write_image('progress_figures/trial_time_all_cohort0.png')

Separate mice by female and male, calculate fading ensembles for both groups.

In [ ]:
## Set mouse list, path, and z_thresh for binarization
across_time = False
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
mouse_trial_times = {}
## Loop through each mouse
for mouse in tqdm(young_female_mice):
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    trial_times = {}
    ## Load assemblies
    for session in session_list:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, dpath, session)
        ## Set activation values as act
        act = assemblies.activations.values
        ## Load a specific session's behavior data
        if not across_time:
            behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
            aligned_behavior = pd.read_feather(behav_file)
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = aligned_behavior.trials
            ## Get length of time for each trial
            time_diff = []
            for trial in np.unique(trials):
                ## Subset aligned_behavior by a given trial
                behavior = aligned_behavior.loc[trials == trial]
                ## Get the first and last timestamp to determine the window
                first_timestamp, last_timestamp = behavior.t.to_numpy()[0], behavior.t.to_numpy()[-1]
                ## Convert from ms to s
                first_timestamp = first_timestamp / 1000
                last_timestamp = last_timestamp / 1000
                ## Append to time_diff list
                time_diff.append(last_timestamp - first_timestamp)
            trial_times[session] = time_diff
        ## This is where the data gets binned either by even time intervals or by trials
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold = None, x_bin_size = x_bin_size, analysis_type = 'max', zscored = True, alpha = 'sidak')  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'all', z_threshold = None)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends
    mouse_trial_times[mouse] = trial_times

Create a scatterplot of trial length vs max or mean activation strength.

In [15]:
mouse_results = {}
for mouse in young_mice:
    session_dictionary = {}
    for session in session_list:
        behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
        aligned_behavior = pd.read_feather(behav_file)
        ## Load assemblies
        assemblies = ica.load_session_assemblies(mouse, dpath, session)
        act = assemblies['activations'].values
        results = {'trial': [], 'trial_length': [], 'max': [], 'mean': []}
        for trial in np.unique(aligned_behavior.trials):
            behavior = aligned_behavior[aligned_behavior.trials == trial]
            first, last = (behavior.t.to_numpy()[0]/1000), (behavior.t.to_numpy()[-1]/1000) ## convert to seconds
            trial_length = last - first
            ## Get max and mean of activation strength within a trial
            trial_activation = act[:, aligned_behavior.trials == trial]
            results['max'].append(np.mean(np.nanmax(trial_activation, axis = 1)))
            results['mean'].append(np.mean(np.mean(trial_activation, axis = 1)))
            ## Append trial number and trial length
            results['trial'].append(trial)
            results['trial_length'].append(trial_length)
        session_dictionary[session] = results
    mouse_results[mouse] = session_dictionary   
## Create dataframe
trial_df = pd.DataFrame()
for mouse in young_mice:
    for session in session_list:
        df = pd.DataFrame(mouse_results[mouse][session])
        df.insert(4, 'mouse', mouse)
        df.insert(5, 'session', session)
        trial_df = pd.concat([trial_df, df], ignore_index = True)

In [16]:
trial_df

,trial,trial_length,max,mean,mouse,session
0,0,66.128,314.652752,9.661928,Fornax,Training1
1,1,89.968,297.708501,6.272573,Fornax,Training1
2,2,29.871,72.692849,2.555749,Fornax,Training1
3,3,44.831,133.676073,4.249668,Fornax,Training1
4,4,322.959,171.662987,1.860975,Fornax,Training1
...,...,...,...,...,...,...
1575,22,79.793,215.549053,3.009515,Titania,Reversal1
1576,23,89.745,254.499505,3.215564,Titania,Reversal1
1577,24,63.376,187.508708,3.208593,Titania,Reversal1
1578,25,78.992,228.698217,3.272543,Titania,Reversal1


In [39]:
## Linear regression
lm_max = pg.linear_regression(trial_df.trial_length, trial_df['max'], as_dataframe = False)
lm_mean = pg.linear_regression(trial_df.trial_length, trial_df['mean'], as_dataframe = False)
## Plot max
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Max Activation Strength')
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = trial_df['max'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = lm_max['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.show()

In [38]:
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Mean Activation Strength')
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = trial_df['mean'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = lm_mean['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.show()

In [41]:
## Linear regression for JUST REVERSAL SESSION
reversal_df = trial_df[trial_df.session == 'Reversal1']
lm_max_reversal = pg.linear_regression(reversal_df.trial_length, reversal_df['max'], as_dataframe = False)
lm_mean_reversal = pg.linear_regression(reversal_df.trial_length, reversal_df['mean'], as_dataframe = False)
fig = pf.custom_graph_template('Reversal1', x_title = 'Trial Length (s)', y_title = 'Max Activation Strength')
fig.add_trace(go.Scatter(x = reversal_df.trial_length, y = reversal_df['max'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = reversal_df.trial_length, y = lm_max_reversal['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.show()

The code block below is used to replicate the PCA/ICA results from one of Will's mice.

In [ ]:
mouse = 'Fornax'
session = 'Reversal1'
minian_path = pjoin(dpath, 'processed/{}_{}.nc'.format(mouse, session))
minian_data = xr.open_dataset(minian_path)
## Detect assemblies
neural_data = minian_data.S_bin.values
smoothed_data = ctn.moving_average(neural_data, ksize = 5)
assemblies = ica.find_assemblies(smoothed_data, nullhyp = 'circ', n_shuffles = 500)
act = assemblies['activations']
## Load will's PCA/ICA result for Atlas Reversal1
dpath = pjoin(dpath, 'assemblies')
assemblies_will = ica.load_session_assemblies(mouse, dpath, session)
## Load behavior data
behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
aligned_behavior = pd.read_feather(behav_file)
## Get which timestamps are part of which trial for the aligned behavior data
trials = aligned_behavior.trials
## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
determined_trends = {}
trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'all', z_threshold = None)
## Create dictionary for Reversal1 trends
determined_trends[session] = trends
## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
proportion_dict = ica.calculate_proportions_ensembles(determined_trends)